In [2]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import DonutProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from warnings import filterwarnings
from transformers import VisionEncoderDecoderModel, AutoTokenizer
filterwarnings('ignore')
from tqdm import tqdm
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import torch
import pandas as pd
from PIL import Image
from transformers import DonutProcessor, VisionEncoderDecoderModel
from tqdm import tqdm
import requests
from io import BytesIO

In [4]:
# Path to your checkpoint directory
checkpoint_path = "/home/arjun/Desktop/Github/AmazonML-Hackathon/Donut_tests/donut-finetuned-docvqa/checkpoint-5500"

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
# Load the processor from the checkpoint
# processor = DonutProcessor.from_pretrained(checkpoint_path)

# Determine the device to use
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model from the checkpoint and move it to the appropriate device
model = VisionEncoderDecoderModel.from_pretrained(checkpoint_path).to(device)

In [15]:
def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

def perform_inference(image, question):
    # Prepare the input
    encoding = processor(images=image, text=question, return_tensors="pt")
    
    # Move input to the correct device
    for k, v in encoding.items():
        encoding[k] = v.to(device)
    print('start giving outputs')
    # Generate the output
    outputs = model.generate(
        pixel_values=encoding["pixel_values"],
        # decoder_input_ids=encoding["decoder_input_ids"],
        max_length=model.config.max_length,
        early_stopping=True,
        pad_token_id=model.config.pad_token_id,
        eos_token_id=model.config.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

    # Decode the output
    sequence = processor.batch_decode(outputs.sequences)[0]
    sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    sequence = sequence.strip()

    return sequence

In [8]:
test_df = pd.read_csv('/home/arjun/Desktop/Github/AmazonML-Hackathon/dataset/test.csv')

# Define the entity_unit_map (as in your training script)
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}


In [ ]:
def get_local_image_path(image_link, image_folder):
    # Extract filename from the URL
    filename = image_link.split('/')[-1]
    # Construct the full path to the local image file
    local_path = os.path.join(image_folder, filename)
    return local_path

image_folder = '/home/arjun/Desktop/Github/AmazonML-Hackathon/images/test_images'


In [ ]:
results = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_path = get_local_image_path(row['image_link'], image_folder)
    
    if row['entity_name'] in entity_unit_map:
        potential_units = entity_unit_map[row['entity_name']]
        question = f"Given the image, what is the {row['entity_name']} of the item in {potential_units}: "
    else:
        question = f"Given the image, what is the {row['entity_name']}: "
    
    print(f"Processing image: {image_path}")
    print(f"Question: {question}")
    
    prediction = perform_inference(image_path, question)
    results.append({'index': row['index'], 'prediction': prediction})

In [ ]:
output_df = pd.DataFrame(results)

# Save the results to a CSV file
output_df.to_csv('inference_results.csv', index=False)

print("Inference completed. Results saved to 'inference_results.csv'")

In [16]:
# results = []
# for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
#     image_link = download_image(row['image_link'])
#     img_files = os.listdir('/home/arjun/Desktop/Github/AmazonML-Hackathon/images/test_images')
#     if row['entity_name'] in entity_unit_map:
#         potential_units = entity_unit_map[row['entity_name']]
#         question = f"Given the image, what is the {row['entity_name']} of the item in {potential_units}: "
#     else:
#         question = f"Given the image, what is the {row['entity_name']}: "
#     print(image, question)
#     prediction = perform_inference(image, question)
#     results.append({'index': row['index'], 'prediction': prediction})


  0%|          | 0/131187 [00:00<?, ?it/s]

<PIL.Image.Image image mode=RGB size=500x500 at 0x77265D250970> Given the image, what is the height of the item in {'yard', 'metre', 'foot', 'millimetre', 'inch', 'centimetre'}: 
start giving outputs


  0%|          | 1/131187 [00:01<58:04:12,  1.59s/it]

<PIL.Image.Image image mode=RGB size=1000x65 at 0x77265D24FDF0> Given the image, what is the width of the item in {'yard', 'metre', 'foot', 'millimetre', 'inch', 'centimetre'}: 
start giving outputs


  0%|          | 2/131187 [00:03<57:34:53,  1.58s/it]

...

  0%|          | 6/131187 [00:09<57:19:55,  1.57s/it]

<PIL.Image.Image image mode=RGB size=1000x65 at 0x77265C561090> Given the image, what is the width of the item in {'yard', 'metre', 'foot', 'millimetre', 'inch', 'centimetre'}: 
start giving outputs


  0%|          | 7/131187 [00:11<57:39:55,  1.58s/it]

<PIL.Image.Image image mode=RGB size=500x500 at 0x77265C5A5720> Given the image, what is the height of the item in {'yard', 'metre', 'foot', 'millimetre', 'inch', 'centimetre'}: 


KeyboardInterrupt: 

In [17]:
results

[{'index': 0, 'prediction': '<s><s> 100.0 gram'},
 {'index': 1, 'prediction': '<s><s> 100.0 gram'},
 {'index': 2, 'prediction': '<s><s> 100.0 gram'},
 {'index': 3, 'prediction': '<s><s> 100.0 gram'},
 {'index': 4, 'prediction': '<s><s> 100.0 gram'},
 {'index': 5, 'prediction': '<s><s> 100.0 gram'},
 {'index': 6, 'prediction': '<s><s> 100.0 gram'}]